# Predicting Procurement Compliance Using KPI-Driven Machine Learning Models

**Author:** Brittany Dowdle

**Date:** 7/4/2025

**Objective:** This notebook is part of the overall [capstone project](https://github.com/Bdowdle4/Dowdle_Analytics_Capstone). The ultimate goal is to identify patterns and predictors of non-compliant suppliers using machine learning techniques.

## Introduction
This project uses the [Procurement KPI Analysis Dataset](https://www.kaggle.com/datasets/shahriarkabir/procurement-kpi-analysis-dataset) to predict compliance of suppliers. The dataset includes purchase order records from 5 different suppliers from 2022-2023. To ensure data quality and model readiness, this notebook will clean and preprocess the dataset to prepare for exporatory analysis and predictive modeling. 

****

### Imports
In the code cell below are the necessary Python libraries for this notebook. *All imports should be at the top of the notebook.*

In [1]:
import pandas as pd
import numpy as np

****

### 1. Load and Inspect the Data

In [2]:
# Load the dataset
df = pd.read_csv("C:/Users/Britt/Documents/44688/Procurement KPI Analysis Dataset.csv")

# Preview the structure and details
print("Dataset Info:")
print(df.info(), '\n')

print("Dataset Description:")
print(df.describe(), '\n')

print("First Few Rows:")
print(df.head(), '\n')

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PO_ID             777 non-null    object 
 1   Supplier          777 non-null    object 
 2   Order_Date        777 non-null    object 
 3   Delivery_Date     690 non-null    object 
 4   Item_Category     777 non-null    object 
 5   Order_Status      777 non-null    object 
 6   Quantity          777 non-null    int64  
 7   Unit_Price        777 non-null    float64
 8   Negotiated_Price  777 non-null    float64
 9   Defective_Units   641 non-null    float64
 10  Compliance        777 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 66.9+ KB
None 

Dataset Description:
          Quantity  Unit_Price  Negotiated_Price  Defective_Units
count   777.000000  777.000000        777.000000       641.000000
mean   1094.660232   58.283822         53.66072

### 2. Standardize Column Names

In [3]:
# Convert all column names to lowercase and remove white space
df.columns = df.columns.str.strip().str.lower()

# View column names
print(df.columns)

Index(['po_id', 'supplier', 'order_date', 'delivery_date', 'item_category',
       'order_status', 'quantity', 'unit_price', 'negotiated_price',
       'defective_units', 'compliance'],
      dtype='object')


### 3. Convert Dates to Datetime Objects

In [4]:
# Parse date fields
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['delivery_date'] = pd.to_datetime(df['delivery_date'], errors='coerce')

# Verify Dtype changed
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   po_id             777 non-null    object        
 1   supplier          777 non-null    object        
 2   order_date        777 non-null    datetime64[ns]
 3   delivery_date     690 non-null    datetime64[ns]
 4   item_category     777 non-null    object        
 5   order_status      777 non-null    object        
 6   quantity          777 non-null    int64         
 7   unit_price        777 non-null    float64       
 8   negotiated_price  777 non-null    float64       
 9   defective_units   641 non-null    float64       
 10  compliance        777 non-null    object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 66.9+ KB
None


### 4. Handle Missing Values
Some purchase orders marked as “Delivered” are missing their delivery_date. Instead of dropping these rows, which would result in a loss of 87 records, this project will use a data-preserving approach. It will combine supplier-specific median lead time imputation with a transparency flag to maintain data integrity.

In [5]:
# Fill missing defective_units with 0 (assuming no defects were found if null)
df['defective_units'] = df['defective_units'].fillna(0)

# Verify null count changed
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777 entries, 0 to 776
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   po_id             777 non-null    object        
 1   supplier          777 non-null    object        
 2   order_date        777 non-null    datetime64[ns]
 3   delivery_date     690 non-null    datetime64[ns]
 4   item_category     777 non-null    object        
 5   order_status      777 non-null    object        
 6   quantity          777 non-null    int64         
 7   unit_price        777 non-null    float64       
 8   negotiated_price  777 non-null    float64       
 9   defective_units   777 non-null    float64       
 10  compliance        777 non-null    object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 66.9+ KB
None


In [6]:
# Calculate lead_time_days for rows with valid delivery_date
df['lead_time_days'] = (df['delivery_date'] - df['order_date']).dt.days

# Compute median lead time by supplier (only using valid delivery_date)
median_lead_times = df.dropna(subset=['lead_time_days']).groupby('supplier')['lead_time_days'].median()

# Create a column to flag missing delivery_date with status 'Delivered'
df['delivery_date_missing_flag'] = df['delivery_date'].isnull() & (df['order_status'].str.lower() == 'delivered')

# Convert flag column to integer
df['delivery_date_missing_flag'] = df['delivery_date_missing_flag'].astype(int)
# Results: 1 for True, 0 for False

# Impute delivery_date using order_date + median lead time per supplier
def impute_delivery_date(row):
    if row['delivery_date_missing_flag']:
        supplier = row['supplier']
        median_days = median_lead_times.get(supplier, df['lead_time_days'].median())
        return row['order_date'] + pd.Timedelta(days=median_days)
    return row['delivery_date']

df['delivery_date'] = df.apply(impute_delivery_date, axis=1)

# Recalculate lead_time_days after imputation
df['lead_time_days'] = (df['delivery_date'] - df['order_date']).dt.days

# Show the first few rows to verify changes
print(df.head())


      po_id         supplier order_date delivery_date    item_category  \
0  PO-00001        Alpha_Inc 2023-10-17    2023-10-25  Office Supplies   
1  PO-00002  Delta_Logistics 2022-04-25    2022-05-05  Office Supplies   
2  PO-00003         Gamma_Co 2022-01-26    2022-02-15              MRO   
3  PO-00004    Beta_Supplies 2022-10-09    2022-10-28        Packaging   
4  PO-00005  Delta_Logistics 2022-09-08    2022-09-20    Raw Materials   

  order_status  quantity  unit_price  negotiated_price  defective_units  \
0    Cancelled      1176       20.13             17.81              0.0   
1    Delivered      1509       39.32             37.34            235.0   
2    Delivered       910       95.51             92.26             41.0   
3    Delivered      1344       99.85             95.52            112.0   
4    Delivered      1180       64.07             60.53            171.0   

  compliance  lead_time_days  delivery_date_missing_flag  
0        Yes             8.0                 

### 5. Create Engineered Features

In [7]:
# Price difference
df['price_diff'] = df['unit_price'] - df['negotiated_price']

# Defect rate
df['defect_rate'] = df['defective_units'] / df['quantity']
df['defect_rate'] = df['defect_rate'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Show first few rows to verify new columns
print(df.head())

      po_id         supplier order_date delivery_date    item_category  \
0  PO-00001        Alpha_Inc 2023-10-17    2023-10-25  Office Supplies   
1  PO-00002  Delta_Logistics 2022-04-25    2022-05-05  Office Supplies   
2  PO-00003         Gamma_Co 2022-01-26    2022-02-15              MRO   
3  PO-00004    Beta_Supplies 2022-10-09    2022-10-28        Packaging   
4  PO-00005  Delta_Logistics 2022-09-08    2022-09-20    Raw Materials   

  order_status  quantity  unit_price  negotiated_price  defective_units  \
0    Cancelled      1176       20.13             17.81              0.0   
1    Delivered      1509       39.32             37.34            235.0   
2    Delivered       910       95.51             92.26             41.0   
3    Delivered      1344       99.85             95.52            112.0   
4    Delivered      1180       64.07             60.53            171.0   

  compliance  lead_time_days  delivery_date_missing_flag  price_diff  \
0        Yes             8.0    

### 6. Identify Outliers
If less than 10% of rows contain outliers, assume it is a natural variance and leave as is. 

In [8]:
# Use the IQR method to determine which values are outliers
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outlier_mask = (data[column] < lower_bound) | (data[column] > upper_bound)
    
    print(f"{column}: {outlier_mask.sum()} outliers")
    return outlier_mask

# Choose columns to search in
outlier_columns = ['quantity', 'unit_price', 'price_diff', 'defect_rate']

for col in outlier_columns:
    detect_outliers_iqr(df, col)

# Provide count of rows with outliers for each column
df['quantity_outlier'] = detect_outliers_iqr(df, 'quantity')
df['unit_price_outlier'] = detect_outliers_iqr(df, 'unit_price')
df['price_diff_outlier'] = detect_outliers_iqr(df, 'price_diff')
df['defect_rate_outlier'] = detect_outliers_iqr(df, 'defect_rate')

# Total number of rows with any outliers
df['any_outlier'] = df[['quantity_outlier', 'unit_price_outlier',
                        'price_diff_outlier', 'defect_rate_outlier']].any(axis=1)
print(f"Total rows with at least one outlier: {df['any_outlier'].sum()}", '\n')

# Show percentage of rows with outliers
outliers = 15
total_rows = 777

percentage = (outliers / total_rows) * 100
print(f"Outliers: {outliers} of {total_rows} rows ({percentage:.2f}%)")


quantity: 6 outliers
unit_price: 0 outliers
price_diff: 8 outliers
defect_rate: 1 outliers
quantity: 6 outliers
unit_price: 0 outliers
price_diff: 8 outliers
defect_rate: 1 outliers
Total rows with at least one outlier: 15 

Outliers: 15 of 777 rows (1.93%)


****
All initial cleaning tasks are completed. Going to verify missing values were all filled before exporting the cleaned data set.

### 7. Second Check for Missing Values

In [9]:
# Check for remaining nulls
df.isnull().sum()

po_id                          0
supplier                       0
order_date                     0
delivery_date                 19
item_category                  0
order_status                   0
quantity                       0
unit_price                     0
negotiated_price               0
defective_units                0
compliance                     0
lead_time_days                19
delivery_date_missing_flag     0
price_diff                     0
defect_rate                    0
quantity_outlier               0
unit_price_outlier             0
price_diff_outlier             0
defect_rate_outlier            0
any_outlier                    0
dtype: int64

In [10]:
# Check the order status for remaining null values
df[df['delivery_date'].isnull()]['order_status'].value_counts()

order_status
Cancelled              8
Pending                6
Partially Delivered    5
Name: count, dtype: int64

### 8. Drop Remaining Rows Containing Null Values
To maintain data integrity and ensure that only relevant, interpretable records are used in lead time calculations and compliance modeling, this project is dropping the 19 rows where: 8 are cancelled orders, 6 the order has not shipped yet, and 5 fulfillment is incomplete. Including these records could introduce bias or noise into the analysis.

In [11]:
# Define list of statuses where missing delivery_date is not "delivered"
drop_statuses = ['cancelled', 'pending', 'partially delivered']

# Drop rows with missing delivery_date AND status in drop_statuses
df = df[~((df['delivery_date'].isnull()) & (df['order_status'].str.lower().isin(drop_statuses)))]

# Final check for null values
df.isnull().sum()  # Should all be 0 now

po_id                         0
supplier                      0
order_date                    0
delivery_date                 0
item_category                 0
order_status                  0
quantity                      0
unit_price                    0
negotiated_price              0
defective_units               0
compliance                    0
lead_time_days                0
delivery_date_missing_flag    0
price_diff                    0
defect_rate                   0
quantity_outlier              0
unit_price_outlier            0
price_diff_outlier            0
defect_rate_outlier           0
any_outlier                   0
dtype: int64

****
### Conclusion
Export cleaned data set as a CSV to be used in exporatory data analysis next.

In [12]:
# Save cleaned dataset
df.to_csv(r"C:\Users\Britt\Documents\44688\Dowdle_Analytics_Capstone\Data\cleaned_procurement_data.csv", index=False)